In [ ]:
import numpy as np
import scipy.constants as spc
import pandas as pd
import altair as alt

In [ ]:
# Constants
mu_0 = spc.mu_0
pi = spc.pi
mag_const = mu_0/(4*pi)

In [ ]:
D=pd.read_table("DATALOG.TXT",names=['date','time','position','voltage'])
pos=pd.DataFrame({'point':['begin','end'],'latitude':[47.250755, 47.250771],'longitude':[-122.480139,-122.480191]})
print(pos)
D['magnetic_field']=D['voltage'].str.replace(' uV','').astype(float)/(4*12.2*100)
D.loc[D['magnetic_field']>0,'latitude']=np.linspace(pos['latitude'][0],pos['latitude'][1],len(D['date'][D['magnetic_field']>0]))
D['magnetic_anomaly']=D['magnetic_field']-D.loc[117,'magnetic_field']
print(D)

In [ ]:
# Plot anomaly field
measured_plot=alt.Chart(D.loc[D['magnetic_field']>0,:]).mark_point(color='steelblue',shape='square').encode(
    x=alt.X("latitude:Q",scale=alt.Scale(domain=(pos['latitude'][0],pos['latitude'][1]))),
    y=alt.Y("magnetic_anomaly:Q",scale=alt.Scale(domain=(-2,2)))
)
display(measured_plot)

In [ ]:
# Set up survey parameters
D = 100   # Line length in m
h = 1      # Station height in m above ground
N=1000     # Number of points

survey_line=pd.DataFrame({'x':[np.array(xi) for xi in np.vstack([np.linspace(-D,D,N),np.ones((N,))*h]).T]})

In [ ]:
# Set up Earth's magnetic field
latitude = 47 # Latitude
I_Earth = np.arctan(2*np.tan(np.radians(latitude)))  # Inclination of Earth's field
B_Earth = 50E-6  # Earth's field magnitude in T (Note: B field, need to divide by mu_0 to get H)

In [ ]:
# Set up source parameters

yd = 0.     # Depth of dipole in m
I_dipole = np.radians(80) # Inclination of dipole in degrees (assume N pole is on +X side)
    # Set equal to I_Earth for induced magnetization
mu_magnitude = 1. # Dipole moment magnitude in Am^2

# Calculate dipole moment vector
mu = mu_magnitude*np.array([np.cos(I_dipole), -np.sin(I_dipole)])

def dipole(x_survey, x_dipole, mu):
    r = x_survey-x_dipole
    r_magnitude = np.linalg.norm(r)
    r_hat = r/r_magnitude
    return mag_const*3*((np.dot(mu,r_hat)*r_hat)-mu)/(r_magnitude**3)
    

survey_line['B'] = survey_line['x'].apply(dipole,x_dipole=np.array([0, -yd]), mu=mu)
print(survey_line)

In [ ]:
# Calculate anomaly field
survey_line['delta_B']=survey_line['B'].apply(lambda xi: np.dot(xi,np.array([B_Earth*np.cos(np.radians(I_Earth)), 
                                                                             -B_Earth*np.sin(np.radians(I_Earth))])))

In [ ]:
# Break vectors x and B into components for plotting
survey_line['transect_distance']=survey_line['x'].apply(lambda xi: xi[0])
survey_line['Bh']=survey_line['B'].apply(lambda xi: xi[0])
survey_line['Bv']=survey_line['B'].apply(lambda xi: xi[1])

In [ ]:
# Plot anomaly field
anomaly_plot=alt.Chart(survey_line).mark_line().encode(
    x=alt.X("transect_distance:Q",scale=alt.Scale(domain=(-100,100))),
    y=alt.Y("delta_B:Q")
)
display(anomaly_plot)